In [16]:
import os, sys, glob
import numpy as np
import pandas as pd

#### read AU and generate label.csv

In [46]:
AU = ["AU%02d" % (i+1) for i in range(99)]
df = pd.DataFrame(columns=['ID', 'subject', 'task', 'frame_no']+AU)
for val in glob.glob('./AU_OCC/*.csv'):
    tmp = pd.read_csv(val, skiprows=1, names=['frame_no', *AU])
    subject, task = val.split('/')[-1][:-4].split('_')
    tmp.insert(loc=0, column='subject', value=subject)
    tmp.insert(loc=1, column='task', value=task)
    def fun(row): 
        return '_'.join(row.values.astype(np.str))
    tmp.insert(loc=0, column='ID', value=tmp[['subject', 'task', 'frame_no']].apply(func=fun, axis=1))
    df = df.append(tmp)

In [47]:
df.head()

,ID,subject,task,frame_no,AU01,AU02,AU03,AU04,AU05,AU06,...,AU90,AU91,AU92,AU93,AU94,AU95,AU96,AU97,AU98,AU99
0,M006_T5_648,M006,T5,648,0,0,9,0,0,0,...,9,9,9,9,9,9,9,9,9,9
1,M006_T5_649,M006,T5,649,0,0,9,0,0,0,...,9,9,9,9,9,9,9,9,9,9
2,M006_T5_650,M006,T5,650,0,0,9,0,0,0,...,9,9,9,9,9,9,9,9,9,9
3,M006_T5_651,M006,T5,651,0,0,9,0,0,0,...,9,9,9,9,9,9,9,9,9,9
4,M006_T5_652,M006,T5,652,0,0,9,0,0,0,...,9,9,9,9,9,9,9,9,9,9


In [55]:
df.to_csv('./AU_OCC/all_au.csv', index=False)

In [6]:
# 9为缺失值，
""" 这些AU不缺失
['AU01', 'AU02', 'AU04', 'AU05', 'AU06', 'AU07',
'AU09', 'AU10', 'AU11', 'AU12', 'AU13', 'AU14', 'AU15', 'AU16', 'AU17',
'AU18', 'AU19', 'AU20', 'AU22', 'AU23', 'AU24', 'AU27', 'AU28', 'AU29',
'AU30', 'AU31', 'AU32', 'AU33', 'AU34', 'AU35', 'AU36', 'AU37', 'AU38',
'AU39']
如同实验室惯例，使用的AU为 [AU01, AU02, AU04, AU06, AU07, AU10, AU12, AU14, AU15, AU17, AU23, AU24]
""" 
select_cols = ['ID', 'subject', 'task', 'frame_no'] + \
    ['AU01', 'AU02', 'AU04', 'AU06', 'AU07', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'AU23', 'AU24']
select_df = df[select_cols]
select_df.head()

,ID,subject,task,frame_no,AU01,AU02,AU04,AU06,AU07,AU10,AU12,AU14,AU15,AU17,AU23,AU24
0,M006_T5_648,M006,T5,648,0,0,0,0,0,0,0,1,0,1,0,0
1,M006_T5_649,M006,T5,649,0,0,0,0,0,0,0,1,0,1,1,0
2,M006_T5_650,M006,T5,650,0,0,0,0,0,0,0,1,0,1,1,0
3,M006_T5_651,M006,T5,651,0,0,0,0,0,0,0,1,0,1,1,0
4,M006_T5_652,M006,T5,652,0,0,0,0,0,0,0,1,0,1,1,0


In [7]:
def fun(x):
    sub, task, frame = x.split('_')
    frame = "%04d.jpg" % (int(frame)-1)
    return '/'.join([sub, task, frame])
select_df.insert(loc=1, column='path', value=select_df['ID'].map(fun))

In [8]:
map_ = {
    'T1': 'happy', 'T2':'sadness', 'T3': 'surprise', 'T4': 'embarrassment', \
    'T5':'fear', 'T6':'pain', 'T7':'anger', 'T8':'disgust'
}
select_df.insert(loc=2, column='expression', value=select_df['task'].map(map_))

In [9]:
select_df.expression.value_counts()

sadness          20837
fear             20795
anger            20339
embarrassment    20208
happy            20074
pain             19553
surprise         12821
disgust          12220
Name: expression, dtype: int64

In [10]:
select_df.head()

,ID,path,expression,subject,task,frame_no,AU01,AU02,AU04,AU06,AU07,AU10,AU12,AU14,AU15,AU17,AU23,AU24
0,M006_T5_648,M006/T5/0647.jpg,fear,M006,T5,648,0,0,0,0,0,0,0,1,0,1,0,0
1,M006_T5_649,M006/T5/0648.jpg,fear,M006,T5,649,0,0,0,0,0,0,0,1,0,1,1,0
2,M006_T5_650,M006/T5/0649.jpg,fear,M006,T5,650,0,0,0,0,0,0,0,1,0,1,1,0
3,M006_T5_651,M006/T5/0650.jpg,fear,M006,T5,651,0,0,0,0,0,0,0,1,0,1,1,0
4,M006_T5_652,M006/T5/0651.jpg,fear,M006,T5,652,0,0,0,0,0,0,0,1,0,1,1,0


In [11]:
# 各个AU的发生频率
select_df[['AU01', 'AU02', 'AU04', 'AU06', 'AU07', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'AU23', 'AU24']].sum(axis=0)/select_df.shape[0]

AU01    0.211390
AU02    0.170994
AU04    0.202626
AU06    0.460861
AU07    0.548980
AU10    0.594292
AU12    0.562014
AU14    0.465621
AU15    0.169353
AU17    0.343255
AU23    0.165397
AU24    0.151375
dtype: float64

In [12]:
select_df.to_csv('./label.csv', index=False)

#### judge whether the corresponding images exist

In [13]:
import tqdm
import pandas as pd
import os

In [14]:
df = pd.read_csv('./label.csv')
df.head()

,ID,path,expression,subject,task,frame_no,AU01,AU02,AU04,AU06,AU07,AU10,AU12,AU14,AU15,AU17,AU23,AU24
0,M006_T5_648,M006/T5/0647.jpg,fear,M006,T5,648,0,0,0,0,0,0,0,1,0,1,0,0
1,M006_T5_649,M006/T5/0648.jpg,fear,M006,T5,649,0,0,0,0,0,0,0,1,0,1,1,0
2,M006_T5_650,M006/T5/0649.jpg,fear,M006,T5,650,0,0,0,0,0,0,0,1,0,1,1,0
3,M006_T5_651,M006/T5/0650.jpg,fear,M006,T5,651,0,0,0,0,0,0,0,1,0,1,1,0
4,M006_T5_652,M006/T5/0651.jpg,fear,M006,T5,652,0,0,0,0,0,0,0,1,0,1,1,0


In [15]:
img_dir = './Images'
df['exist'] = False
for index in tqdm.tqdm_notebook(df.index):
    path = '/'.join([img_dir, df.loc[index, 'path']])
    df.loc[index, 'exist'] = os.path.exists(path)

In [18]:
real_df = df[df.exist==True]
real_df.shape, df.shape

((146573, 19), (146847, 19))

In [19]:
real_df.to_csv('./label_real.csv', index=False)